# COMP5318 Assignment 1: Rice Classification

##### Group number: ...
##### Student 1 SID: ...
##### Student 2 SID: ...  
##### Student 3 SID: ... 
##### Student 4 SID: ... 

In [15]:
# Import all libraries
from sklearn.model_selection import StratifiedKFold
from warnings import simplefilter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy import signal
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [16]:
# Ignore future warnings
simplefilter(action='ignore', category=FutureWarning)

In [17]:
# Load the rice dataset: rice-final2.csv

pd.set_option('display.max_columns', None)
df = pd.read_csv('test-before.csv')


In [18]:
# Pre-process dataset

# Keep X and y as DataFrames/Series first
X = df.iloc[:, :-1].copy()
y = df.iloc[:, -1].copy()

# Clean X
for col in X.columns:
    X[col] = X[col].replace(r"[^0-9.\-]", np.nan, regex=True)
    X[col] = pd.to_numeric(X[col], errors="coerce")

# Handle missing values
imputer = SimpleImputer(strategy="mean")
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# ===== STEP 2: Normalize to [0,1] =====
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X_imputed.columns)

# ===== STEP 3: Map classes =====
y_mapped = (
    y.replace({"class1": 0, "class2": 1, "Class1": 0, "Class2": 1})
     .pipe(pd.to_numeric, errors="coerce")
     .fillna(0)
     .astype(int)
)

# ===== Merge result =====
df_processed = pd.concat([X_scaled, y_mapped], axis=1)

# Round and export
df_processed = df_processed.astype(float).round(4)
df_processed.to_csv("test-after.csv", index=False, float_format="%.4f")

df_processed.head()


,a1,a2,a3,a4,a5,a6,class
0,0.0621,0.4999,0.5410,0.2079,0.2594,0.0613,0.0
1,0.8073,0.7474,0.6721,0.2634,0.2038,0.0586,0.0
2,0.3105,0.6030,0.4187,0.0000,0.0000,0.0900,0.0
3,0.3105,0.5618,0.6148,0.3604,0.0000,0.0950,0.0
4,0.1863,0.8144,0.6230,0.4990,0.4539,0.1597,1.0


In [19]:
# Print first ten rows of pre-processed dataset to 4 decimal places as per assignment spec
# A function is provided to assist

def print_data(X, y, n_rows=10):
    """Takes a numpy data array and target and prints the first ten rows.
    
    Arguments:
        X: numpy array of shape (n_examples, n_features)
        y: numpy array of shape (n_examples)
        n_rows: numpy of rows to print
    """
    for example_num in range(n_rows):
        for feature in X[example_num]:
            print("{:.4f}".format(feature), end=",")

        if example_num == len(X)-1:
            print(y[example_num],end="")
        else:
            print(y[example_num])
            


In [20]:
X = df_processed.iloc[:, :-1].values
y = df_processed.iloc[:, -1].values
print_data(X,y,10)

0.0621,0.4999,0.5410,0.2079,0.2594,0.0613,0.0
0.8073,0.7474,0.6721,0.2634,0.2038,0.0586,0.0
0.3105,0.6030,0.4187,0.0000,0.0000,0.0900,0.0
0.3105,0.5618,0.6148,0.3604,0.0000,0.0950,0.0
0.1863,0.8144,0.6230,0.4990,0.4539,0.1597,1.0
0.1863,0.6039,0.4754,0.1525,0.1000,0.0655,1.0
0.6832,0.7114,0.6230,0.0000,0.0000,0.0877,1.0
0.5589,0.5258,0.6230,0.5129,0.0000,0.0869,0.0
0.1242,0.4639,0.5574,0.5822,0.0000,0.1009,1.0
0.2484,0.5722,0.5902,0.6515,0.3835,0.0979,0.0


### Part 1: Cross-validation without parameter tuning

In [21]:
## Setting the 10 fold stratified cross-validation
cvKFold=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# The stratified folds from cvKFold should be provided to the classifiers

In [22]:
# Logistic Regression
def logregClassifier(X, y):

    clf = LogisticRegression(random_state=0, max_iter=1000)
    scores = cross_val_score(clf, X, y, cv=cvKFold)
   
    return scores.mean()

In [23]:
#Naïve Bayes

def nbClassifier(X, y):
    model = GaussianNB()
    scores = cross_val_score(model, X, y, cv=cvKFold)
    
    return scores.mean()

In [24]:
# Decision Tree
def dtClassifier(X, y):
    clf = DecisionTreeClassifier(criterion="entropy", random_state=0)
    scores = cross_val_score(clf, X, y, cv=cvKFold)
    
    return scores.mean()

In [25]:
# Ensembles: Bagging, Ada Boost and Gradient Boosting
def bagDTClassifier(X, y, n_estimators, max_samples, max_depth):
    base_dt = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth, random_state=0)
    bagging = BaggingClassifier(
        estimator=base_dt,
        n_estimators=n_estimators,
        max_samples=max_samples,
        random_state=0
    )
    scores = cross_val_score(bagging, X, y, cv=cvKFold)
    return scores.mean()

def adaDTClassifier(X, y, n_estimators, learning_rate, max_depth):
    base_dt = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth, random_state=0)
    ada = AdaBoostClassifier(
        estimator=base_dt,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=0
    )
    scores = cross_val_score(ada, X, y, cv=cvKFold)
    return scores.mean()

def gbClassifier(X, y, n_estimators, learning_rate):
    gb = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=0
    )
    scores = cross_val_score(gb, X, y, cv=cvKFold)
    return scores.mean()

### Part 1 Results

In [26]:
# Parameters for Part 1:

#NB
X = df_processed.iloc[:, :-1].values
y = df_processed.iloc[:, -1].values
naive_score = nbClassifier(X, y)

#Bagging
bag_n_estimators = 50
bag_max_samples = 100
bag_max_depth = 5

bag_score = bagDTClassifier(X, y, bag_n_estimators, bag_max_samples, bag_max_depth)

#AdaBoost
ada_n_estimators = 50
ada_learning_rate = 0.5
ada_bag_max_depth = 5

ada_score = adaDTClassifier(X, y, ada_n_estimators, ada_learning_rate, ada_bag_max_depth)

#GB
gb_n_estimators = 50
gb_learning_rate = 0.5

gb_score = gbClassifier(X, y, gb_n_estimators, gb_learning_rate)

#logR
lr_mean = logregClassifier(X, y)

#DT
dt_mean = dtClassifier(X, y)

# Print results for each classifier in part 1 to 4 decimal places here:
print("LogR average cross-validation accuracy: ",lr_mean)
print("NB average cross-validation accuracy: ",naive_score)
print("DT average cross-validation accuracy: ",dt_mean)
print("Bagging average cross-validation accuracy: ",bag_score)
print("AdaBoost average cross-validation accuracy: ",ada_score)
print("GB average cross-validation accuracy: ",gb_score)

LogR average cross-validation accuracy:  0.67
NB average cross-validation accuracy:  0.6554761904761905
DT average cross-validation accuracy:  0.7895238095238095
Bagging average cross-validation accuracy:  0.7514285714285713
AdaBoost average cross-validation accuracy:  0.7702380952380953
GB average cross-validation accuracy:  0.712857142857143


### Part 2: Cross-validation with parameter tuning

In [27]:
# KNN

k = [1, 3, 5, 7]
p = [1, 2]

def bestKNNClassifier(X, y):
    # use global/public k and p defined above
    param_grid = {'n_neighbors': k, 'p': p}

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=0, stratify=y
    )

    gs = GridSearchCV(KNeighborsClassifier(), param_grid,
                      cv=10, scoring='accuracy')
    gs.fit(X_train, y_train)

    best_k = gs.best_params_['n_neighbors']
    best_p = gs.best_params_['p']
    cv_acc = gs.best_score_

    test_acc = accuracy_score(y_test, gs.best_estimator_.predict(X_test))

    return best_k, best_p, cv_acc, test_acc

In [28]:
# Random Forest
# You should use RandomForestClassifier from sklearn.ensemble with information gain and max_features set to ‘sqrt’.
n_estimators = [10, 30, 60, 100]
max_leaf_nodes = [6, 12]

def bestRFClassifier(X, y):
# ===== Split data into train/test with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=0
    )

    # ===== Base Random Forest model 
    rf = RandomForestClassifier(
        criterion="entropy", # use information gain
        max_features="sqrt", # feature selection strategy
        random_state=0,
    )

    # ====== Parameter grid 
    param_grid = {
        "n_estimators": n_estimators,  
        "max_leaf_nodes": max_leaf_nodes,   
    }

    # ===== Grid search with cross-validation 
    gscv = GridSearchCV(
        estimator=rf,
        param_grid=param_grid,
        scoring="accuracy",
        cv=cvKFold,
        n_jobs=1,      
        refit=True,
        return_train_score=False,
    )
    gscv.fit(X_train, y_train)

    # ===== Get best model and CV score 
    best_model = gscv.best_estimator_
    best_params = gscv.best_params_
    best_cv_acc = float(gscv.best_score_)

    # ====== Evaluate on the test set 
    y_pred = best_model.predict(X_test)
    test_acc = float(accuracy_score(y_test, y_pred))
    macro_f1 = float(f1_score(y_test, y_pred, average="macro"))
    weighted_f1 = float(f1_score(y_test, y_pred, average="weighted"))

    return (
        int(best_params["n_estimators"]),
        int(best_params["max_leaf_nodes"]),
        best_cv_acc,
        test_acc,
        macro_f1,
        weighted_f1
    )


### Part 2: Results

In [29]:
# Perform Grid Search with 10-fold stratified cross-validation (GridSearchCV in sklearn). 
# The stratified folds from cvKFold should be provided to GridSearchV

# This should include using train_test_split from sklearn.model_selection with stratification and random_state=0
# Print results for each classifier here. All results should be printed to 4 decimal places except for
# "k", "p", n_estimators" and "max_leaf_nodes" which should be printed as integers.

best_k, best_p, cv_score, test_score = bestKNNClassifier(X, y)

print("KNN best k:", best_k)
print("KNN best p:", best_p)
print("KNN cross-validation accuracy:", cv_score)
print("KNN test set accuracy:", test_score)

print()

best_n, best_leaf, cv_acc, tst_acc, macro_f1, weighted_f1 = bestRFClassifier(X, y)


print("RF best n_estimators:", best_n)
print("RF best max_leaf_nodes:", best_leaf)
print("RF cross-validation accuracy:", round(cv_acc, 4))
print("RF test set accuracy:", round(tst_acc, 4))
print("RF test set macro average F1:", round(macro_f1, 4))
print("RF test set weighted average F1:", round(weighted_f1, 4))

KNN best k: 1
KNN best p: 1
KNN cross-validation accuracy: 0.7176470588235293
KNN test set accuracy: 0.5952380952380952

RF best n_estimators: 60
RF best max_leaf_nodes: 12
RF cross-validation accuracy: 0.8202
RF test set accuracy: 0.6667
RF test set macro average F1: 0.6541
RF test set weighted average F1: 0.6635
